In [1]:
# install singular value decomposition
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [2]:
# import tools
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf

from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.evaluation.single_table import evaluate_quality
from google.colab import drive
import logging
import warnings
drive.mount('/content/gdrive')
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, log_loss, f1_score, confusion_matrix, roc_auc_score, precision_score, recall_score
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier

Mounted at /content/gdrive


In [3]:
# data processing
df = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/Database_Features_ML.xlsx')
print(f"Original dataframe dimensions: {df.shape}")
df[df.columns[4:]] = df[df.columns[4:]].apply(pd.to_numeric, errors='coerce')
df.replace(["#!NULL", ""], np.nan, inplace=True)
df.drop(columns=['ID Paziente'], inplace=True)
df = df.dropna()
print(f"Processed dataframe dimensions: {df.shape}")

# encoding sex column
label_encoder = LabelEncoder()
df['Sesso'] = label_encoder.fit_transform(df['Sesso'])
# encoding disease column
df['Disease'] = df['Disease'].replace({'ALS': 1, 'HEALTHY': 0})
df_als = df[df['Disease'] == 1]
df_hc = df[df['Disease'] == 0]

print(f"Number of ALS: {len(df_als)}")
print(f"Number of HC: {len(df_hc)}")
print('-'*50)
print(df.columns)
feature_names = df.columns.tolist()  # Store feature names

Original dataframe dimensions: (134, 38)
Processed dataframe dimensions: (118, 37)
Number of ALS: 97
Number of HC: 21
--------------------------------------------------
Index(['Sesso', 'Disease', 'Età', 'GCSF', 'IFNgamma', 'IL10', 'IL15', 'IL17A',
       'IL1beta', 'IL2', 'IL4', 'IL6', 'IL8', 'MCP1', 'MIP1alfa', 'TNFalfa',
       'VEGF', 'TTVlog', 'TTVcopies', 'acetic', 'Propionic', 'Butyric',
       'isoButyric', 'isoValeric', '@MethylButyric', 'valeric', 'Hexanoic',
       'Heptanoic', 'Nonanoic', '@EthylHexanoic', 'Octanoic', 'Decanoic',
       'Benzoic', 'Dodecanoic', 'Tetradecanoic', 'Hexadecanoic',
       'Octadecanoic'],
      dtype='object')


In [4]:
# subsampling the majority class, ALS
df_als_downsized = df_als.sample(n=21, random_state=42)
df_downsized = pd.concat([df_als_downsized, df_hc], ignore_index=True)
# spliting data into training 50% and testing 50%
class_a = df_downsized[df_downsized['Disease'] == 1]
class_b = df_downsized[df_downsized['Disease'] == 0]
half_class_a = class_a.sample(frac=0.5, random_state=42)
half_class_b = class_b.sample(frac=0.5, random_state=42)
train_df = pd.concat([half_class_a, half_class_b], axis=0).reset_index(drop=True)
test_df = pd.concat([class_a.drop(half_class_a.index), class_b.drop(half_class_b.index)], axis=0).reset_index(drop=True)
print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

# Save to CSV
test_df.to_csv("/content/gdrive/MyDrive/ALS/real.csv", index=False)

Train set shape: (20, 37)
Test set shape: (22, 37)


In [5]:
# Detect metadata and ensure 'Disease' is categorical
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_df)
metadata.update_column('Disease', sdtype='categorical')

# validate
metadata.validate()
metadata.validate_data(data=train_df)

# Initialize and fit the synthesizer
synthesizer_GC = GaussianCopulaSynthesizer(
        metadata,  # required
        enforce_min_max_values=True,
        enforce_rounding=False,
        default_distribution='gaussian_kde'
        )
synthesizer_GC.fit(train_df)

# Generate synthetic data
# Sample 1 (10xTrainingSet)
synthetic_data = synthesizer_GC.sample(num_rows=10*len(train_df))
print(synthetic_data.shape)
# Save to CSV
synthetic_data.to_csv("/content/gdrive/MyDrive/ALS/synthetic.csv", index=False)

(200, 37)


In [6]:
quality_report = evaluate_quality(train_df, synthetic_data, metadata)
print(quality_report)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 37/37 [00:00<00:00, 468.71it/s]|
Column Shapes Score: 80.28%

(2/2) Evaluating Column Pair Trends: |██████████| 666/666 [00:14<00:00, 47.38it/s]|
Column Pair Trends Score: 92.76%

Overall Score (Average): 86.52%



In [7]:
# Spliting features and labels

Xtrain = synthetic_data.drop(columns=['Disease']).values  # Features
ytrain = synthetic_data['Disease'].values  # Labels

# Use the test set
Xtest = test_df.drop(columns=['Disease']).values  # Features
ytest = test_df['Disease'].values  # Labels

print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(200, 36)
(200,)
(22, 36)
(22,)


In [8]:
# keras

# Scale the data for improved training stability
scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(ytrain), y=ytrain)
class_weight_dict = dict(enumerate(class_weights))

# Define the deep learning model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(96, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Cross-validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Metrics containers
val_accuracies = []
val_log_losses = []
val_f1_scores = []
val_precisions = []
val_recalls = []
val_specificities = []
val_aurocs = []

print("Starting cross-validation...")

for train_index, val_index in kf.split(X_train, ytrain):
    print(f"\nTraining fold {fold}...")

    # Split training data into train and validation sets
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]

    # Create and train the model
    model = create_model(X_train_fold.shape[1])

    # Define callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

    history = model.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        epochs=100,
        batch_size=32,
        verbose=1,
        callbacks=[early_stopping, lr_scheduler],
        class_weight=class_weight_dict
    )

    # Evaluate the model on the validation set
    y_val_pred_proba = model.predict(X_val_fold)
    y_val_pred_classes = (y_val_pred_proba > 0.5).astype(int)

    # Calculate metrics
    val_accuracy = accuracy_score(y_val_fold, y_val_pred_classes)
    val_log_loss = log_loss(y_val_fold, y_val_pred_proba)
    val_f1 = f1_score(y_val_fold, y_val_pred_classes)
    val_precision = precision_score(y_val_fold, y_val_pred_classes)
    val_recall = recall_score(y_val_fold, y_val_pred_classes)
    val_auroc = roc_auc_score(y_val_fold, y_val_pred_proba)

    # Specificity calculation
    cm = confusion_matrix(y_val_fold, y_val_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    val_specificity = tn / (tn + fp)

    # Append metrics
    val_accuracies.append(val_accuracy)
    val_log_losses.append(val_log_loss)
    val_f1_scores.append(val_f1)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_specificities.append(val_specificity)
    val_aurocs.append(val_auroc)

    print(f"Fold {fold} - Val Accuracy: {val_accuracy:.4f}, Val Log Loss: {val_log_loss:.4f}, "
          f"Val F1: {val_f1:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, "
          f"Val Specificity: {val_specificity:.4f}, Val AUROC: {val_auroc:.4f}")
    fold += 1

# Final Cross-Validation Results
print("\nCross-Validation Results:")
print(f"Average Val Accuracy: {np.mean(val_accuracies):.4f}")
print(f"Average Val Log Loss: {np.mean(val_log_losses):.4f}")
print(f"Average Val F1 Score: {np.mean(val_f1_scores):.4f}")
print(f"Average Val Precision: {np.mean(val_precisions):.4f}")
print(f"Average Val Recall: {np.mean(val_recalls):.4f}")
print(f"Average Val Specificity: {np.mean(val_specificities):.4f}")
print(f"Average Val AUROC: {np.mean(val_aurocs):.4f}")

# Save the modelprint()
model.save("/content/gdrive/MyDrive/ALS/dl.keras")

y_test_pred_proba = model.predict(X_test)
y_test_pred_classes = (y_test_pred_proba > 0.5).astype(int)

test_accuracy = accuracy_score(ytest, y_test_pred_classes)
test_log_loss = log_loss(ytest, y_test_pred_proba)
test_f1 = f1_score(ytest, y_test_pred_classes)
test_precision = precision_score(ytest, y_test_pred_classes)
test_recall = recall_score(ytest, y_test_pred_classes)
test_auroc = roc_auc_score(ytest, y_test_pred_proba)

cm_test = confusion_matrix(ytest, y_test_pred_classes)
tn_test, fp_test, fn_test, tp_test = cm_test.ravel()
test_specificity = tn_test / (tn_test + fp_test)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Log Loss: {test_log_loss:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test Specificity: {test_specificity:.4f}")
print(f"Test AUROC: {test_auroc:.4f}")


Starting cross-validation...

Training fold 1...
Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 244ms/step - accuracy: 0.5029 - loss: 0.9025 - val_accuracy: 0.3500 - val_loss: 0.8160 - learning_rate: 1.0000e-04
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4993 - loss: 0.8421 - val_accuracy: 0.3750 - val_loss: 0.8071 - learning_rate: 1.0000e-04
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5359 - loss: 0.7870 - val_accuracy: 0.3750 - val_loss: 0.7971 - learning_rate: 1.0000e-04
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5247 - loss: 0.8967 - val_accuracy: 0.4000 - val_loss: 0.7874 - learning_rate: 1.0000e-04
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5195 - loss: 0.7730 - val_accuracy: 0.4000 - val_loss: 0.7777 - learning_rate: 1.0000e-04
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6052 - loss: 0.7324 - val_accuracy: 0.4750 - val_loss: 0.7685 - learning_rate: 1.0000e-04
Epoch 7/100
5/5 ━━━━━━━━━━

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Fold 3 - Val Accuracy: 0.6750, Val Log Loss: 0.5793, Val F1: 0.6977, Val Precision: 0.6250, Val Recall: 0.7895, Val Specificity: 0.5714, Val AUROC: 0.8471

Training fold 4...
Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 129ms/step - accuracy: 0.5240 - loss: 0.8232 - val_accuracy: 0.4500 - val_loss: 0.7191 - learning_rate: 1.0000e-04
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4611 - loss: 0.8327 - val_accuracy: 0.4750 - val_loss: 0.7078 - learning_rate: 1.0000e-04
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5520 - loss: 0.7717 - val_accuracy: 0.5000 - val_loss: 0.6963 - learning_rate: 1.0000e-04
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6571 - loss: 0.6904 - val_accuracy: 0.5250 - val_loss: 0.6851 - learning_rate: 1.0000e-04
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5033 - loss: 0.7346 - val_accuracy: 0.5250 - val_loss: 0.6731 - learning_rate: 1.0000e-04
Epoch 6/100
5/5 

In [9]:
# AdaBoostClassifier

# Scale the data for improved stability
scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

# Define the AdaBoost model
model = AdaBoostClassifier(n_estimators=25, learning_rate=0.1, random_state=42)

# Cross-validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Metrics containers
accuracies = []
log_losses = []
f1_scores = []
precisions = []
recalls = []
specificities = []
aurocs = []

print("Starting cross-validation...")

for train_index, val_index in kf.split(X_train, ytrain):
    print(f"\nTraining fold {fold}...")

    # Split training data into train and validation sets
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]

    # Train the AdaBoost classifier
    model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_val_pred_proba = model.predict_proba(X_val_fold)[:, 1]
    y_val_pred_classes = model.predict(X_val_fold)

    # Calculate validation metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred_classes)
    log_loss_value = log_loss(y_val_fold, y_val_pred_proba)
    f1 = f1_score(y_val_fold, y_val_pred_classes)
    precision = precision_score(y_val_fold, y_val_pred_classes)
    recall = recall_score(y_val_fold, y_val_pred_classes)
    auroc = roc_auc_score(y_val_fold, y_val_pred_proba)

    # Specificity calculation
    cm = confusion_matrix(y_val_fold, y_val_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Append metrics
    accuracies.append(accuracy)
    log_losses.append(log_loss_value)
    f1_scores.append(f1)
    precisions.append(precision)
    recalls.append(recall)
    specificities.append(specificity)
    aurocs.append(auroc)

    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Log Loss: {log_loss_value:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, AUROC: {auroc:.4f}")
    fold += 1

# Final aggregated cross-validation results
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Log Loss: {np.mean(log_losses):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average Specificity: {np.mean(specificities):.4f}")
print(f"Average AUROC: {np.mean(aurocs):.4f}")

# Final evaluation on the test set
print("\nFinal Test Evaluation...")
model.fit(X_train, ytrain)  # Train the model on the full training data

# Save the trained AdaBoost model before testing
joblib.dump({"model": model, "feature_names": feature_names}, "/content/gdrive/MyDrive/ALS/adaboost.pkl")


# Test set predictions
y_test_pred_proba = model.predict_proba(X_test)[:, 1]
y_test_pred_classes = model.predict(X_test)

# Calculate test set metrics
test_accuracy = accuracy_score(ytest, y_test_pred_classes)
test_log_loss = log_loss(ytest, y_test_pred_proba)
test_f1 = f1_score(ytest, y_test_pred_classes)
test_precision = precision_score(ytest, y_test_pred_classes)
test_recall = recall_score(ytest, y_test_pred_classes)
test_auroc = roc_auc_score(ytest, y_test_pred_proba)

# Specificity calculation
cm_test = confusion_matrix(ytest, y_test_pred_classes)
tn, fp, fn, tp = cm_test.ravel()
test_specificity = tn / (tn + fp)

# Print test results
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Log Loss: {test_log_loss:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test Specificity: {test_specificity:.4f}")
print(f"Test AUROC: {test_auroc:.4f}")


Starting cross-validation...

Training fold 1...
Fold 1 - Accuracy: 0.7250, Log Loss: 0.5203, F1: 0.6857, Precision: 0.7500, Recall: 0.6316, Specificity: 0.8095, AUROC: 0.8070

Training fold 2...
Fold 2 - Accuracy: 0.8500, Log Loss: 0.3686, F1: 0.8235, Precision: 0.9333, Recall: 0.7368, Specificity: 0.9524, AUROC: 0.9749

Training fold 3...
Fold 3 - Accuracy: 0.7750, Log Loss: 0.5226, F1: 0.8000, Precision: 0.6923, Recall: 0.9474, Specificity: 0.6190, AUROC: 0.8471

Training fold 4...
Fold 4 - Accuracy: 0.8750, Log Loss: 0.3740, F1: 0.8718, Precision: 0.8500, Recall: 0.8947, Specificity: 0.8571, AUROC: 0.9549

Training fold 5...
Fold 5 - Accuracy: 0.8750, Log Loss: 0.3996, F1: 0.8780, Precision: 0.7826, Recall: 1.0000, Specificity: 0.7727, AUROC: 0.9482

Cross-Validation Results:
Average Accuracy: 0.8200
Average Log Loss: 0.4370
Average F1 Score: 0.8118
Average Precision: 0.8016
Average Recall: 0.8421
Average Specificity: 0.8022
Average AUROC: 0.9064

Final Test Evaluation...
Test Accu

In [10]:
# LGBMClassifier

# Scale the data for improved stability
scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

# Define model parameters
model = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=-1,
    random_state=42
)

# Cross-validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Metrics containers
cv_accuracies = []
cv_log_losses = []
cv_f1_scores = []
cv_precisions = []
cv_recalls = []
cv_specificities = []
cv_aurocs = []

print("Starting cross-validation...")

fold = 1
for train_index, val_index in kf.split(X_train, ytrain):
    print(f"\nTraining fold {fold}...")

    # Split data into train and validation sets
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]

    # Scale the data within the fold
    scaler = StandardScaler()
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)

    # Train the model
    model.fit(X_train_fold, y_train_fold)

    # Predict on the validation set
    y_val_pred_proba = model.predict_proba(X_val_fold)[:, 1]
    y_val_pred_classes = model.predict(X_val_fold)

    # Calculate validation metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred_classes)
    log_loss_value = log_loss(y_val_fold, y_val_pred_proba)
    f1 = f1_score(y_val_fold, y_val_pred_classes)
    precision = precision_score(y_val_fold, y_val_pred_classes)
    recall = recall_score(y_val_fold, y_val_pred_classes)
    auroc = roc_auc_score(y_val_fold, y_val_pred_proba)

    # Specificity calculation
    cm = confusion_matrix(y_val_fold, y_val_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Append metrics
    cv_accuracies.append(accuracy)
    cv_log_losses.append(log_loss_value)
    cv_f1_scores.append(f1)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_specificities.append(specificity)
    cv_aurocs.append(auroc)

    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Log Loss: {log_loss_value:.4f}, F1: {f1:.4f}, "
          f"Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, AUROC: {auroc:.4f}")
    fold += 1

# Final Cross-Validation Results
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(cv_accuracies):.4f}")
print(f"Average Log Loss: {np.mean(cv_log_losses):.4f}")
print(f"Average F1 Score: {np.mean(cv_f1_scores):.4f}")
print(f"Average Precision: {np.mean(cv_precisions):.4f}")
print(f"Average Recall: {np.mean(cv_recalls):.4f}")
print(f"Average Specificity: {np.mean(cv_specificities):.4f}")
print(f"Average AUROC: {np.mean(cv_aurocs):.4f}")

# Final Test Evaluation
print("\nFinal Test Evaluation...")

# Train model on the entire training set
model.fit(X_train, ytrain)

# Save the model
joblib.dump({"model": model, "feature_names": df}, "/content/gdrive/MyDrive/ALS/lgbm.pkl")


# Evaluate on the test set
y_test_pred_proba = model.predict_proba(X_test)[:, 1]
y_test_pred_classes = model.predict(X_test)

test_accuracy = accuracy_score(ytest, y_test_pred_classes)
test_log_loss = log_loss(ytest, y_test_pred_proba)
test_f1 = f1_score(ytest, y_test_pred_classes)
test_precision = precision_score(ytest, y_test_pred_classes)
test_recall = recall_score(ytest, y_test_pred_classes)
test_auroc = roc_auc_score(ytest, y_test_pred_proba)

cm_test = confusion_matrix(ytest, y_test_pred_classes)
tn, fp, fn, tp = cm_test.ravel()
test_specificity = tn / (tn + fp)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Log Loss: {test_log_loss:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test Specificity: {test_specificity:.4f}")
print(f"Test AUROC: {test_auroc:.4f}")


Starting cross-validation...

Training fold 1...
[LightGBM] [Info] Number of positive: 75, number of negative: 85
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001369 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1488
[LightGBM] [Info] Number of data points in the train set: 160, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.468750 -> initscore=-0.125163
[LightGBM] [Info] Start training from score -0.125163
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu